# $CO_2$ emissions and historical temperature change

In this project I will look at historical temperature changes and try to establish whether these depend on the level of pollution for various countries. First we are going to look at the historical $CO_2$ emissions data, downloaded from [global carbon atlas](http://www.globalcarbonatlas.org/en/CO2-emissions). The data for 2015 and 2016 is preliminary.

In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
from plotly import tools
plotly.offline.init_notebook_mode(connected=True)
from scipy import stats

## $CO_2$ emission levels per country
First we load the historical $CO_2$ emission levels table. It shows emission levels per country over time.

In [3]:
co2_data = pd.read_csv('CO2_historical_1.csv')
co2_data = co2_data.rename(columns= {'Unnamed: 0': 'Year'})
co2_data.set_index('Year', inplace=True, drop=True)
co2_data.head()

,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
Year,,,,,,,,,,,,,,,,,,,,,
1960,0.41403,2.0225,6.1555,NaN,0.54960,NaN,0.036640,48.7752,2.4946,0.61856,...,4.3162,47.8664,NaN,57.0228,7.4856,NaN,NaN,3.6310,4.3561,5.9446
1961,0.49098,2.2790,6.0603,NaN,0.45434,NaN,0.047632,51.1384,2.5726,0.64555,...,4.1183,49.3630,NaN,51.8859,7.9802,NaN,NaN,2.6637,3.7096,5.0623
1962,0.68883,2.4622,5.6645,NaN,1.17980,NaN,0.102590,53.6520,2.7000,0.70894,...,4.0084,51.8088,0.040304,54.0623,9.3395,NaN,NaN,3.8838,3.5833,4.8899
1963,0.70715,2.0812,5.4227,NaN,1.15050,NaN,0.084272,50.0429,2.8956,0.67909,...,4.3162,55.5615,0.032976,56.1581,9.1124,NaN,NaN,2.9165,3.4450,4.7013
1964,0.83906,2.0152,5.6462,NaN,1.22380,NaN,0.091600,55.6818,3.0795,0.66028,...,4.5544,59.0898,0.062288,56.5575,11.7908,NaN,NaN,3.6310,3.2756,4.4701


Next we want to look at how many countries contribute to certain percentage of emissions. We will look at the data for 2014 as the most recent year for which data has been confirmed. As a preliminary cleaning, countries for which emission levels for 2014 are not given, have been removed.

In [5]:
mask = np.array(pd.notnull(co2_data.loc[[2014],:]).values)[0,:] 
new_columns = np.array(co2_data.columns.tolist())[mask].tolist()
co2_data_clean = co2_data.loc[:,new_columns]
co2_data_clean.head()

,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,...,United States of America,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Wallis and Futuna Islands,Yemen,Zambia,Zimbabwe
Year,,,,,,,,,,,,,,,,,,,,,
1960,0.41403,2.0225,6.1555,NaN,0.54960,NaN,0.036640,48.7752,2.4946,0.61856,...,2888.3312,4.3162,47.8664,NaN,57.0228,7.4856,NaN,3.6310,4.3561,5.9446
1961,0.49098,2.2790,6.0603,NaN,0.45434,NaN,0.047632,51.1384,2.5726,0.64555,...,2878.1489,4.1183,49.3630,NaN,51.8859,7.9802,NaN,2.6637,3.7096,5.0623
1962,0.68883,2.4622,5.6645,NaN,1.17980,NaN,0.102590,53.6520,2.7000,0.70894,...,2984.7640,4.0084,51.8088,0.040304,54.0623,9.3395,NaN,3.8838,3.5833,4.8899
1963,0.70715,2.0812,5.4227,NaN,1.15050,NaN,0.084272,50.0429,2.8956,0.67909,...,3116.6790,4.3162,55.5615,0.032976,56.1581,9.1124,NaN,2.9165,3.4450,4.7013
1964,0.83906,2.0152,5.6462,NaN,1.22380,NaN,0.091600,55.6818,3.0795,0.66028,...,3253.3316,4.5544,59.0898,0.062288,56.5575,11.7908,NaN,3.6310,3.2756,4.4701


Below, we define a function which accepts as an argument the number of countries with highest emissions and returns what percentage of the total world's emissions they represent.This function is then used to create a table with two columns: number of countries with highest emission levels and the percentage they contribute. 

In [6]:
sorted_2014 = co2_data_clean.loc[[2014],:].sort_values(axis=1,by=[2014], ascending=False)
world_emission_2014 = sorted_2014.iloc[0,:].values.sum()
def percentage_emissions(n):
    '''Takes the number of highest n contributos and returns what percentage of the global CO_2 emissions the contribute'''
    big_n = sorted_2014.iloc[0,0:n].values.sum()
    return (big_n/world_emission_2014)*100

In [7]:
number_highes_emission = np.arange(1,sorted_2014.shape[1]+1)

percentege_emissions_2014=pd.DataFrame({
       'number of countries': number_highes_emission
   }
)

percentege_emissions_2014['percentage_co2'] = percentege_emissions_2014['number of countries'].apply(percentage_emissions)
percentege_emissions_2014.head()

,number of countries,percentage_co2
0,1,29.572802
1,2,45.577778
2,3,52.009526
3,4,56.810887
4,5,60.453317


That dependence is shown on the plot below. For example, we can see that the biggest 40 contributors make up 92% of the total CO_2 emission. The biggest 80 contributors make up 98% of the world's CO_2 emission.

In [8]:
trace0 = go.Scatter(
    x = percentege_emissions_2014['number of countries'],
    y = percentege_emissions_2014['percentage_co2'],
    mode = 'markers',
    name = 'markers'
)

data = [trace0]

layout= go.Layout(
    title= 'Percentage of the total world emission for the biggest CO_2 contributors',
    xaxis= dict(
        title= 'Number of countries considered',
        ticklen= 5,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Percentage of the total emission',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend= False
)
fig= go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

Next we want to focus at the countries with highest emissions. The plot below shows the top ten contributing countries and percentage they each contribute individually. The hovering feature shows the name of the country and the corresponding percentage.

In [9]:
def percentage_country(emission_country):
    return (emission_country/world_emission_2014)*100
    

data = sorted_2014.loc[[2014],:].values[10:]
df_index = np.arange(0,sorted_2014.loc[[2014],:].values.size).tolist()
CO_2_emissions_2014 = pd.DataFrame({'emissions': sorted_2014.loc[[2014],:].values[0,:],
                                    'countries':sorted_2014.columns.tolist()
                                   }, 
                                   index = df_index)

CO_2_emissions_2014['percentage_country'] = CO_2_emissions_2014['emissions'].apply(percentage_country)

top_ten_percentage = CO_2_emissions_2014['percentage_country'][0:10].values.tolist()
top_ten_percentage.append((sorted_2014.loc[[2014],:].values[0,10:].sum()/world_emission_2014)*100)
top_ten_countries = CO_2_emissions_2014['countries'][0:10].values.tolist()
top_ten_countries.append('Rest of the world')


fig = {
    'data': [
        {
            'values': top_ten_percentage,
            'labels': top_ten_countries,
            'text':'CO2',
            'textposition':'inside',
            'type': 'pie',
            'name': 'CO2 Emissions',
            'hoverinfo':'label+percent+name'
        }
    ],
    'layout':{
        'title':'Global CO2 Emissions 2014'
        
    }
}


plotly.offline.iplot(fig)


## Historical temperatures per country

Next we look at historical data for average temperature at different countries. There is a table for each individual country, which needs to be downloaded manually (any suggestion about how this could be done automatically are very welcome). The data is from [global carbon atlas](http://www.globalcarbonatlas.org/en/CO2-emissions). The function, defined below will create a table, containing temperatures  for a given country. These are monthly temperatures for the period 1900 - 2016.


In [11]:
months = np.arange(1,13)
years = np.arange(1905,2016)

def create_country_table(country_name, years):
    '''Creates a table with temperatures over time, when name of country and years are supplied'''
    country_table_yearly=pd.DataFrame({
       'Month':months
    })
    
    country_table = pd.read_csv(country_name+'_temp.csv')
    country_table.rename(columns={'\tYear': 'Year', ' Month': 'Month'}, inplace=True)
    for i in years:
        monthly_temps = country_table.loc[(country_table['Year']==i), ['tas','Month']].reset_index(drop=True)
        monthly_temps.rename(columns={'tas': i}, inplace=True)
        country_table_yearly = pd.merge(country_table_yearly, monthly_temps, on=['Month'])
    return (country_table_yearly)

We choose to look at four different countries. Two of them with very big $CO_2$ emission, China and USA, and the other two with very low emission levels, Central African Republic and Andorra. Below we create a list of tables for the countries we are interested in and make a dictionary out of that. The dictionary contains the countries as key and tables as data.

In [12]:
countries = ['china','usa','caf','andorra']
list_tables = [create_country_table(country,years) for country in countries ]
countries_dict= dict(zip(countries,list_tables))

For example, the table for China looks a follows:

In [13]:
countries_dict['china'].head(5)

,Month,1905,1906,1907,1908,1909,1910,1911,1912,1913,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,1,-7.46430,-9.46770,-7.67190,-8.96220,-9.25280,-9.00460,-9.03020,-8.4983,-8.99520,...,-7.69890,-6.84520,-9.29150,-7.29470,-7.26700,-10.82300,-9.80020,-8.55500,-6.63980,-6.32350
1,2,-6.96130,-7.68660,-7.00330,-6.53090,-5.89580,-6.49830,-6.23210,-4.7335,-6.25310,...,-3.93230,-1.76460,-6.82070,-2.76560,-4.87130,-3.86770,-6.06670,-4.80000,-5.61260,-3.27160
2,3,-1.39290,-0.09490,-0.53760,-0.65120,-0.83000,-0.92920,-0.80160,-0.5594,-0.90290,...,1.63290,1.80544,3.62850,1.81753,0.72168,-0.13800,0.70142,2.59283,2.37766,2.58015
3,4,5.83789,7.45592,7.18896,7.14655,7.11037,6.68162,7.36569,7.4724,6.59303,...,8.13849,8.73790,8.90717,9.59543,6.56534,8.92701,9.10929,7.90616,9.26856,8.70064
4,5,12.36590,12.75290,12.68650,12.77510,12.73210,12.81680,12.61900,12.9012,12.34680,...,13.85010,14.68740,14.16820,13.84900,13.58680,13.56230,14.42420,14.08140,10.73510,13.91400


Below we plot the average monthly temperatures for several years. For the plots the library `plotly` is used. 
Every figure is a `json` object, containing dictionary with all data about the plots. 

In [11]:
  countries[0]

'china'

In [14]:
Years_to_plot = [1910, 1945, 1990, 2015]
trace_dict={}
colors = ['yellow', 'green', 'red', 'blue']

#For each country in the given above list generate the x and y coordinates for the scatter plot.
#The coordinates for each country are stored in a the list trace.
for country in countries:
    table=countries_dict[country]    
    x_data = table['Month'].values.tolist()
    
    
    trace = []
    for index,year in enumerate(Years_to_plot):
        trace.append(go.Scatter(                     
            x=x_data,
            y=table[year].values.tolist(),
            name = year,
            mode = 'lines',
           line=dict(color=colors[index])
        )
    )
    trace_dict[country] = trace 

    layout = go.Layout(
       xaxis=dict(
           showline=True
       )
    )  
fig = tools.make_subplots(rows=2, cols=2, subplot_titles = ('China', 'United States of America', 'Central African Republic', 'Andorra' ) )

for i in np.arange(len(trace_dict['china'])):
    fig.append_trace(trace_dict['china'][i], 1, 1)
for i in np.arange(len(trace_dict['usa'])):
    fig.append_trace(trace_dict['usa'][i], 1, 2)   
for i in np.arange(len(trace_dict['caf'])):
    fig.append_trace(trace_dict['caf'][i], 2, 1)   
for i in np.arange(len(trace_dict['caf'])):
    fig.append_trace(trace_dict['andorra'][i], 2, 2)   

fig['layout'].update(showlegend=False, title='Monthly temperatures')

plotly.offline.iplot(fig, filename='make-subplots-multiple-with-titles')


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



There is no legend as the plot get too busy, but the year appears when the hovering feature is used. It can be seen from these plots that the average temperature increases slightly with time as the blue curve is from 2015 and it is slightly higher on average than the other (red: 1990, green:1945, yellow:1910).

## Temperature change for different levels of $CO_2$ emissions

Now we would like to look at the yearly temperature change for a given country for certain months. We are interested to see if there is a significant difference between the countries with high and low levels of $CO_2$ emissions. 

In [15]:
months_names = {
    '1': 'January',
    '2': 'February',
    '3': 'March',
    '4': 'April',
    '5': 'May',
    '6': 'June',
    '7': 'July',
    '8': 'August',
    '9': 'September',
   '10':'October',
   '11':'November',
   '12':'December'
}

 Below a function is defined, which will prepare the coordinates for the plot:

* x coordinates: years in a given range
* y coordinates: temperatures over time for each of the selected countries for every month needed.

They are stored in the dictionary data_trace. Keys of the dictionary are the countries and the coordinates are the data.

At present, the code works for two months only. This could easily be extended and generalized. The plotting facility expects a list of four countries, this part would be more difficult to extend as the layout would need of the plot would need to change. 

In [16]:
def plot_coordinates(months, countries):
    ''' Calculating the x and y coordinates for the temperature over time for a given time'''
    line_country={}
    x_data = years
    data_trace = {}
    
    for index, country in enumerate(countries):
        #grouping was done to split the countries into ones that get their legend printed
        #and others who don't, to avoid redundant labeling. This did not work, for some reason.
 
        #getting the coordinates for the first month 
        month = int(months_to_plot[0])
        country_table=countries_dict[country]     
        trace_0 =[]
        line_month = {}
        corr_month = country_table[country_table['Month']==month]   #select the row, corresponding to the month needed
        if index ==0:
            groupindex = 'group1',                         
            showlegend_group = 'True'
        else:
            groupindex = 'group2',
            showlegend_group = 'False'
    
        trace_0=go.Scatter(
            x = x_data,
            y = corr_month.loc[:,(corr_month.columns.isin(years))].values.tolist()[0],
            name=months_names[str(month)],
            legendgroup = groupindex[0],
            showlegend = showlegend_group,
            line=dict(color='red')
        )
        #For each onth, fit the data with a linear fit. Record to slopes in the list   line_month for each month
        slope, intercept, r_value, p_value, std_err = stats.linregress(x_data, trace_0['y'])
        line_month[ month ]=[slope,intercept]
    
    
      #getting the coordinates for the second month 
        month = int(months_to_plot[1])
        trace_1 =[]
        corr_month = country_table[country_table['Month']==month]
        trace_1 = go.Scatter(
            x = x_data,
            y = corr_month.loc[:,(corr_month.columns.isin(years))].values.tolist()[0],
            name=months_names[str(month)],
            legendgroup=groupindex[0],
            showlegend = showlegend_group,
            line=dict(color='blue')
        )
        slope, intercept, r_value, p_value, std_err = stats.linregress(x_data, trace_1['y'])
        line_month[ month ]=[slope,intercept]
        
        line_country[country]=line_month 
        data_trace[country] = [trace_0, trace_1]
    return data_trace, line_country
    

In [17]:
#parameters and coordinates to plot

data_trace = {}
months_to_plot = ['1','8']
countries =['china','usa','caf','andorra']
data_trace, line_country = plot_coordinates(months_to_plot,countries)

The plot are created below. the coordinates are added for each month and country.

In [18]:
fig = tools.make_subplots(rows=2, cols=2, subplot_titles=(countries[0][0].upper()+countries[0][1:], countries[1][0].upper()+countries[1][1:],
                                                          countries[2][0].upper()+countries[2][1:], countries[3][0].upper()+countries[3][1:]))


fig.append_trace(data_trace[countries[0]][0], 1, 1)
fig.append_trace(data_trace[countries[0]][1], 1, 1)
fig.append_trace(data_trace[countries[1]][0], 1, 2)
fig.append_trace(data_trace[countries[1]][1], 1, 2)
fig.append_trace(data_trace[countries[2]][0], 2, 1)
fig.append_trace(data_trace[countries[2]][1], 2, 1)
fig.append_trace(data_trace[countries[3]][0], 2, 2)
fig.append_trace(data_trace[countries[3]][1], 2, 2)
layout=go.Layout(title="First Plot", xaxis={'title':'x1'}, yaxis={'title':'x2'})
fig.layout.xaxis3.update({'title':'Year'})
fig.layout.yaxis3.update({'title':'Temperature'})

plotly.offline.iplot(fig, filename='Temperature vs. time')


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



Above we also fit the data for different years to a straight line. The slope and intercept for the lines for different countries are as follows:

In [19]:
def print_line(months,countries,line_country):
    line_country
    for country in countries:
        for month in months_to_plot:
            slope,intercept = line_country[country][int(month)][0],line_country[country][int(month)][1]
            intercept =round(intercept,3)
            intercept = '+'+str(intercept) if intercept>0 else str(intercept)
            c = "{a}, {b}:".format(a=country[0].upper()+country[1:],b= months_names[month])
            print("{a:18s}, y= {b:8.5f} {intr:7s}".format(a=c, b=slope, intr=intercept ) )    

In [20]:
print_line(months_to_plot,countries, line_country)

China, January:   , y=  0.01179 -31.744
China, August:    , y=  0.00155 +15.62 
Usa, January:     , y=  0.01049 -26.42 
Usa, August:      , y=  0.00799 +3.356 
Caf, January:     , y= -0.00393 +32.008
Caf, August:      , y=  0.00569 +12.613
Andorra, January: , y=  0.01810 -32.062
Andorra, August:  , y=  0.01504 -9.848 


From the line equations above we can see that Andorra has the highest rate of increase of its temperature, and at the same time one of the lowest $CO_2$ emission rates. This was probably not a good country to pick  for our study as it is neighboring Germany and France, two countries with high emission levels. If we look at the
slopes for January only (which seems consistently higher), we can see that China and USA have higher rate of increase of the temperature: 0.01179 and 0.01049, compared to Caf: -0.00393.
Of course, this is not enough to answer the question whether there is relationship between the $CO_2$ emission levels and the rate of increase of the temperature. 

The next step would be to calculate the rate of increase of the temperature for every country and compare more months, or maybe the average temperature per year. The code could easily be extended to do that, the only difficulty I see is downloading so many files manually. 

Finally, it would be interesting then to make a 2D plot of the rate of increase vs. emission rates and try to investigate whether there are clusters of countries with similar properties, using machine learning algorithms. 